# **<center><font style="color:rgb(100,109,254)">Module 3: Advance Gesture Controlled Shape/Object Manipulation </font></center>**

<center>
    <img src='https://drive.google.com/uc?export=download&id=1OGxEgnz1eeMKP-y9dvYtBfV1Zc5VR1to'>
    <a href='https://www.microsoft.com/en-us/hololens/developers'>HoloLens photo courtesy of Microsoft</a>
</center>


## **<font style="color:rgb(134,19,348)"> Module Outline </font>**

The module can be split into the following parts:


- *Lesson 1: Create a Basic Hand Paint Application*

- *Lesson 2: Add Adjustable Paint Color Functionality*

- ***Lesson 3:* Draw Shapes/Objects utilizing Hand Gestures**  *(This Tutorial)*

- *Lesson 4: Manipulate Shapes/Objects utilizing Hand Gestures*

**Please Note**, these Jupyter Notebooks are not for sharing; do read the Copyright message below the Code License Agreement section, which is in the last cell of this notebook.
-Taha Anwar

Alright, without further ado, let's dive in.

### **<font style="color:rgb(134,19,348)"> Import the Libraries</font>**

First, we will import the required libraries.

In [2]:
import cv2
import math
import numpy as np
import mediapipe as mp
from collections import deque
from scipy.interpolate import interp1d
from previous_lesson import (detectHandsLandmarks, calculateDistance, recognizeGestures)

## **<font style="color:rgb(134,19,348)">Initialize the Hands Landmarks Detection Model</font>**

After that, as we have been doing in the previous lessons, we will need to initialize the **`mp.solutions.hands`** class and then set up the **`mp.solutions.hands.Hands()`** function with appropriate arguments and also initialize **`mp.solutions.drawing_utils`** class that is needed to visualize the detected landmarks. 

In [3]:
# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the Hands functions for videos.
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, 
                       min_detection_confidence=0.8, min_tracking_confidence=0.3)

# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

## **<font style="color:rgb(134,19,348)">Create a Function to Select Shapes</font>**

To draw, we first have to select the shape which we want to draw (in any paint application), as we have several options (like circle, rectangle, square, etc.) to choose from, so we will create a function **`selectShape()`** that will utilize a hand tips landmarks (found using the functions from the previous lessons) to select a shape in real-time. We will define regions in the frames of a webcam feed for each shape, by overlaying a shape selection tab image (shown below) on the top and allowing the user to select a shape by just touching a shape from the tab with his middle fingertip. 

<center>
    <img src='https://drive.google.com/uc?export=download&id=1Z8hDMRdD_sWPtzzVhSBohlOdxBNUL19B' width=600>
</center>

In [4]:
def selectShape(frame, hand_tips_positions, DEFAULT_SHAPE_SIZE): 
    '''
    This function will select a shape utilizing a hand tips landmarks.
    Args:
        frame:               The current frame/image of a real-time webcam feed.
        hand_tips_positions: A dictionary containing the landmarks of the tips of the fingers of a hand.
        DEFAULT_SHAPE_SIZE:  The default size of each drawable shape.
    Returns:
        shape_selected: The shape to draw, selected by the user using middle finger tip.
    '''
    
    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
    
    # Initialize a variable to store the selected shape.
    shape_selected = None
    
    # Read the shapes selection tab image and resize its width equal to the frame width.
    shapes_selector_tab = cv2.imread('media/overlays/shapes_selector.png')
    shapes_selector_height, shapes_selector_width, _ = shapes_selector_tab.shape
    shapes_selector_tab = cv2.resize(shapes_selector_tab, (frame_width, int((frame_width/shapes_selector_width)*shapes_selector_height)))
    
    # Overlay the shape selection tab image on the top of the frame.
    frame[0:shapes_selector_tab.shape[0], 0:shapes_selector_tab.shape[1]] = shapes_selector_tab
    
    # Get the x and y coordinates of tip of the MIDDLE finger of the hand. 
    x, y = hand_tips_positions['MIDDLE']
    
    # Check if the MIDDLE finger tip is over the shape selection tab image.
    if y <= shapes_selector_tab.shape[0]:
        
        # Check if the MIDDLE finger tip is over the Circle ROI.
        if x>(int(frame_width//11.5)-DEFAULT_SHAPE_SIZE//2) and \
        x<(int(frame_width//11.5)+DEFAULT_SHAPE_SIZE//2):
            
            # Update the selected shape variable to 'Circle'.
            shape_selected='Circle'
        
        # Check if the MIDDLE finger tip is over the Polygon ROI.
        elif x>(int(frame_width//4.35)-DEFAULT_SHAPE_SIZE/2) and \
        x<(int(frame_width//4.35)+DEFAULT_SHAPE_SIZE//2):
            
            # Update the selected shape variable to 'Polygon'.
            shape_selected='Polygon'
        
        # Check if the MIDDLE finger tip is over the Rectangle ROI.
        elif x>(int(frame_width//2.37)-DEFAULT_SHAPE_SIZE//2) and \
        x<(int(frame_width//2.37)+DEFAULT_SHAPE_SIZE//2):
            
            # Update the selected shape variable to 'Rectangle'.
            shape_selected='Rectangle'
        
        # Check if the MIDDLE finger tip is over the Square ROI.
        elif x>(int(frame_width//1.64)-DEFAULT_SHAPE_SIZE//2) and \
        x<(int(frame_width//1.64)+DEFAULT_SHAPE_SIZE//2):
            
            # Update the selected shape variable to 'Square'.
            shape_selected='Square'
        
        # Check if the MIDDLE finger tip is over the Triangle ROI.
        elif x>(int(frame_width//1.095)-DEFAULT_SHAPE_SIZE//2) and \
        x<(int(frame_width//1.095)+DEFAULT_SHAPE_SIZE//2):
            
            # Update the selected shape variable to 'Triangle'.
            shape_selected='Triangle'
        
        # Check if the MIDDLE finger tip is over the Right Triangle ROI.
        elif x>(int(frame_width//1.292)-DEFAULT_SHAPE_SIZE//2) and \
        x<(int(frame_width//1.292)+DEFAULT_SHAPE_SIZE//2):
            
            # Update the selected shape variable to 'Right Triangle'.
            shape_selected='Right Triangle'
    
    # Return the selected shape.
    return shape_selected

## **<font style="color:rgb(134,19,348)">Create a Function to Draw Shapes</font>**

Now that we have sorted out the selection part, let's move on to create a function **`drawShapes()`** to draw the selected shape on a webcam feed in real-time. As done in the previous lessons, we will be utilizing a `canvas` image along with a few hand gestures to serve the purpose.

In [5]:
def drawShapes(frame, canvas, shape_selected, hand_gesture, hand_tips_positions, paint_color, shape_size):
    '''
    This function will draw a selected shape on a frame or canvas based on hand gestures.
    Args:
        frame:               The current frame/image of a real-time webcam feed.
        canvas:              A black image equal to the webcam feed size, to draw on.
        shape_selected:      The shape to draw, selected by the user using middle finger tip.
        hand_gesture:        The current hand gesture recognized in the frame.
        hand_tips_positions: A dictionary containing the landmarks of the tips of the fingers of a hand.
        paint_color:         The color to draw shapes with, on the canvas.
        shape_size:          The size of which, the selected shape is to be draw.
    Returns:
        frame:          The frame with the selected shape drawn and current active mode written.
        canvas:         The black image with the selected shapes drawn on it, in the paint color.
        shape_selected: The name of the selected shape.
    '''
    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
    
    # Get the x and y coordinates of tip of the MIDDLE finger of the hand. 
    x, y = hand_tips_positions['MIDDLE']
    
    # Check if the current hand gesture is 'VICTORY'. 
    if hand_gesture == 'VICTORY':
        
        # Write 'Shape Selection Mode Enabled' on the frame.
        cv2.putText(frame, 'Shape Selection Mode Enabled', (10, frame_height-20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, paint_color, 2)
        
        # Store the frame reference in the image variable.
        # Now if we draw on the image (variable), the drawing will be made on the frame.
        image = frame
            
    # Check if the current hand gesture is 'INDEX MIDDLE THUMB POINTING UP'. 
    elif hand_gesture == 'INDEX MIDDLE THUMB POINTING UP':
        
        # Write 'Shape Placement Mode Enabled' on the frame.
        cv2.putText(frame, 'Shape Placement Mode Enabled', (10, frame_height-20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, paint_color, 2)
        
        # Store the canvas reference in the image variable.
        # Now if we draw on the image (variable), the drawing will be made on the canvas.
        image = canvas
           
    # Check if the current hand gesture is 'SPIDERMAN'.
    elif hand_gesture == 'SPIDERMAN':

        # Write 'Clear Everything' on the frame.
        cv2.putText(img=frame, text='Clear Everything', org=(10, frame_height-30), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=paint_color, thickness=2)

        # Clear the canvas, by re-initializing it to a complete black image.
        canvas = np.zeros((frame_height, frame_width, 3), np.uint8)
    
    # Check if the current hand gesture is 'VICTORY' or 'INDEX MIDDLE THUMB POINTING UP'.
    if hand_gesture == 'VICTORY' or hand_gesture == 'INDEX MIDDLE THUMB POINTING UP':
        
        # Check if the selected shape is 'Circle'.
        if shape_selected == 'Circle':
            
            # Get the radius of the circle to draw.
            circle_radius = shape_size//2
            
            # Draw the circle on the image that will be either frame or canvas based on current gesture.
            image = cv2.circle(img=image, center=(x, y), radius=circle_radius,
                               color=paint_color, thickness=-1)
        
        # Check if the selected shape is 'Polygon'.
        elif shape_selected == 'Polygon':
            
            # Get the radius of the polygon to draw.
            polygon_radius = shape_size//2
            
            # Initialize a list to store the polygon contour (boundary) points.
            polygon_pts=[]
            
            # Iterate over 6 times. 
            # As we are drawing a polygon (hexagon) with 6 sides.
            for i in range(6):
                
                # Get the x and y coordinates of a polygon corner.
                poly_x = x + polygon_radius* math.cos(i * 2 * math.pi / 6)
                poly_y = y + polygon_radius * math.sin(i * 2 * math.pi / 6)
                
                # Append the x and y coordinates into the list.
                polygon_pts.append((poly_x, poly_y))
            
            # Draw the polygon on the image that will be either frame or canvas based on current gesture.
            image = cv2.fillPoly(image, pts = [np.array(polygon_pts, np.int32)],
                                 color=paint_color)
        
        # Check if the selected shape is 'Rectangle'.
        elif shape_selected == 'Rectangle':
            
            # Get the rectangle height and width.
            rec_width = shape_size*2
            rec_height = shape_size
            
            # Draw the rectangle on the image that will be either frame or canvas based on current gesture.
            image = cv2.rectangle(image, pt1=(x-rec_width//2,y-rec_height//2),
                                  pt2=(x+rec_width//2,y+rec_height//2),
                                  color=paint_color, thickness=-1)
        
        # Check if the selected shape is 'Square'.
        elif shape_selected == 'Square':
            
            # Draw the square on the image that will be either frame or canvas based on current gesture.
            image = cv2.rectangle(image, pt1=(x-shape_size//2, y-shape_size//2),
                                  pt2=(x+shape_size//2, y+shape_size//2),
                                  color=paint_color, thickness=-1)
        
        # Check if the selected shape is 'Triangle'.
        elif shape_selected == 'Triangle':
            
            # Get the x and y coordinates of the triangle corners.
            triangle_pts= [(x, y-shape_size//2),
                           (x-shape_size//2, y+shape_size//2),
                           (x+shape_size//2, y+shape_size//2)]
            
            # Draw the triangle on the image that will be either frame or canvas based on current gesture.
            image = cv2.drawContours(image=image, contours=[np.array(triangle_pts, np.int32)], contourIdx=0, 
                                     color=paint_color, thickness=-1)
        
        # Check if the selected shape is 'Right Triangle'.
        elif shape_selected == 'Right Triangle':
            
            # Get the x and y coordinates of the right triangle corners.
            triangle_pts= [(x-shape_size//2, y-shape_size//2),
                           (x-shape_size//2, y+shape_size//2),
                           (x+shape_size//2, y+shape_size//2)]
            
            # Draw the right triangle on the image that will be either frame or canvas based on current gesture.
            image = cv2.drawContours(image=image, contours=[np.array(triangle_pts, np.int32)], contourIdx=0, 
                                     color=paint_color, thickness=-1)
    
    # Check if the current hand gesture is 'INDEX MIDDLE THUMB POINTING UP'. 
    if hand_gesture == 'INDEX MIDDLE THUMB POINTING UP':
        
        # Update the selected shape variable to 'None'.
        shape_selected=None
    
    # Return the frame, canvas, and the selected shape.
    return frame, canvas, shape_selected

Now we will utilize the functions **`selectShape()`** and **`drawShapes()`** created above, to draw shapes on a webcam feed using just our hand gestures in real-time.

In [7]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0, cv2.CAP_DSHOW)
camera_video.set(3,1280)
camera_video.set(4,960)

# Get the width of the frame of the webcam video.
frame_width = int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH))

# Create named window for resizing purposes.
cv2.namedWindow('Create Shapes/Objects utilizing Hand Gesture', cv2.WINDOW_NORMAL)

# Initialize a canvas to draw on.
canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                         int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                  dtype=np.uint8)

# Initialize a variable to store the color value.
paint_color = 0, 255, 0

# Initialize a variable to store the hand label for gesture recognition.
hand_label = 'RIGHT'

# Initialize a variable to store the buffer length.
BUFFER_MAX_LENGTH = 5

# Initialize a buffer to store recognized gestures.
buffer = deque([], maxlen=BUFFER_MAX_LENGTH)

# Initialize a constant to store the default shape size. 
DEFAULT_SHAPE_SIZE = 100

# Initialize a variable to store the shape size.
shape_size = DEFAULT_SHAPE_SIZE

# Initialize a variable to store the selected shape. 
shape_selected = None

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
   
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)

    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
    
    # Perform Hands landmarks detection on the frame.
    frame, results = detectHandsLandmarks(frame, hands, draw=True, display=False)
    
    # Check if the hands landmarks in the frame are detected.
    if results.multi_hand_landmarks:
        
        # Perform a hand gesture recognition.
        # I have modified this recognizeGestures() function,
        # to return the fingers tips position of the both hands.
        current_gesture, hands_tips_positions = recognizeGestures(frame, results, hand_label, draw=False,
                                                                  display=False)
        # Check if a known gesture is recognized.
        if current_gesture != 'UNKNOWN':
            
            # Check if all the gestures stored in the buffer are equal to the current gesture.
            if all(current_gesture==gesture for gesture in buffer):
                
                # Append the current gesture into the buffer.
                buffer.append(current_gesture)
                
            # Otherwise.
            else:
                
                # Clear the buffer.
                buffer.clear()
            
            # Check if the length of the buffer is equal to the maxlength.
            if len(buffer) == BUFFER_MAX_LENGTH:
                
                # Check if the current hand gesture is 'VICTORY'.
                if current_gesture == 'VICTORY':
                    
                    # Check if a shape is selected.
                    if shape_selected:
                
                        # Calculate the distance between the middle finger tip and thumb tip landmark of the other hand.
                        distance = calculateDistance(frame, hands_tips_positions['LEFT' if hand_label == 'RIGHT' else 'RIGHT']['MIDDLE'],
                                                     hands_tips_positions['LEFT' if hand_label == 'RIGHT' else 'RIGHT']['THUMB'], display=False)

                        # Check if the distance is calculated successfully.
                        # This will be none in case when the hand is not in the frame.
                        if distance:

                            # Get the interpolation function and calculate the new shape size value.
                            shape_size_interp_f = interp1d([30,230], [50, 200])
                            shape_size = int(shape_size_interp_f(distance))

                            # Write the current distance percentage on the frame.
                            cv2.putText(img=frame, text=f'{int((distance-30)/2)}%',org=(frame_width-130, frame_height-430), 
                                        fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=3, color=paint_color, thickness=3)


                            # Get the interpolation function and calculate the bar value.
                            # This will be used to draw a filled rectangle of height varying with the distance.
                            bar_interp_f = interp1d([30,230],  [frame_height-50, frame_height-400])
                            bar_value = bar_interp_f(distance)  


                            # Draw the filled rectangle with varying height on the frame.
                            cv2.rectangle(frame, (frame_width-80, int(bar_value)), 
                                          (frame_width-50, frame_height-50), (255, 0, 255), -1)

                            # Draw another rectangle around the filled rectangle on the frame.
                            cv2.rectangle(frame, (frame_width-80, frame_height-400),
                                          (frame_width-50, frame_height-50), (0, 255, 0), 6)

                    # Otherwise.
                    else:
                        
                        # Select a shape utilizing hand landmarks.
                        shape_selected = selectShape(frame, hands_tips_positions[hand_label],
                                                     DEFAULT_SHAPE_SIZE)
                
                # Draw the selected shape with center at the tip of the middle finger of the hand.
                frame, canvas, shape_selected = drawShapes(frame, canvas, shape_selected, current_gesture,
                                                           hands_tips_positions[hand_label], paint_color, 
                                                           shape_size)
                                    
    # Otherwise.
    else:
        
        # Clear the buffer.
        buffer.clear()

    # Update the pixel values of the frame with the canvas's values at the indexes where canvas!=0
    # i.e. where canvas is not black and something is drawn there.
    # In short, this will copy the drawings from canvas to the frame.
    frame[np.mean(canvas, axis=2)!=0] = canvas[np.mean(canvas, axis=2)!=0]
    
    # Display the frame.
    cv2.imshow("Create Shapes/Objects utilizing Hand Gesture", frame)
    
   # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF
    
    # Check if 'ESC' is pressed and break the loop.
    if k == 27:
        break
    
    # Check if 's' key is pressed and switch the hand label.
    elif k == ord('s'):
        
        # Set gesture hand label to 'LEFT', if it was 'RIGHT',
        # Otherwise if it was 'LEFT', set it to 'RIGHT'.
        hand_label = 'LEFT' if hand_label == 'RIGHT' else 'RIGHT'
        
# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()



# Additional comments:
#       - In summary, this program will check if the
#         user's selected hand is making the victory
#         gesture while being on the region of interest
#         of the shape selector.
#       - Once the user has selected a shape, the program
#         will draw the shape on the video frame based on 
#         the location of the index and middle finger.
#       - If the user extends his thumb outward while
#         a shape has been selected, then the program will
#         draw the shape on the canvas where it will stay.
#       - On the other hand, the program will stop drawing
#         on the video frame until another shape has been selected.
#       - Apart from that, the user can use his other hand
#         to change the features of the shape that is selected.

Perfect! all the shapes are being drawn as expected. 

### **<font style="color:rgb(255,140,0)"> Code License Agreement </font>**
```
Copyright (c) 2022 Bleedai.com

Feel free to use this code for your own projects commercial or noncommercial, these projects can be Research-based, just for fun, for-profit, or even Education with the exception that you’re not going to use it for developing a course, book, guide, or any other educational products.

Under *NO CONDITION OR CIRCUMSTANCE* you may use this code for your own paid educational or self-promotional ventures without written consent from Taha Anwar (BleedAI.com).

```
